In [57]:
import openai
import langchain
#from pinecone import Pinecone
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.pinecone import Pinecone
from langchain.llms import OpenAI

In [58]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [59]:
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    file_loader.load()
    documents = file_loader.load()
    return documents

In [60]:
documents = read_doc('docs/')
print(len(documents))


50


In [61]:
def chunk_data(documents,chunk_size=800,chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc = text_splitter.split_documents(documents=documents)
    return doc

In [62]:
doc = chunk_data(documents=documents)
print(len(doc))
doc

160


[Document(page_content='Learning OpenCV\nGary Bradski and Adrian Kaehler\nBeijing · Cambridge · Farnham · Köln · Sebastopol · Taipei · Tokyo\nFM-R4886-AT1.indd   iFM-R4886-AT1.indd   i 9/15/08   4:26:38 PM9/15/08   4:26:38 PM', metadata={'source': 'docs\\LearningOpenCV_1_2.pdf', 'page': 2}),
 Document(page_content='Learning OpenCV\nby Gary Bradski and Adrian Kaehler\nCopyright © 2008 Gary Bradski and Adrian Kaehler. All rights reserved.\nPrinted in the United States of America.\nPublished by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions \nare also available for most titles ( safari.oreilly.com ). For more information, contact our \ncorporate/institutional sales department: (800) 998-9938 or corporate@oreilly.com .\nEditor:  Mike Loukides\nProduction Editor:  Rachel Monaghan\nProduction Services:  Newgen Publishing and \nData ServicesCover Designer:  Karen Mo

In [63]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [64]:
for chunk in doc:
    print(chunk.metadata)
    break

{'source': 'docs\\LearningOpenCV_1_2.pdf', 'page': 2}


In [65]:
embedding_function

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [66]:
for chunk in doc:
    #print(chunk.page_content)
    embedded = embedding_function.embed_documents(chunk.page_content)

In [67]:
len(embedded)

519

In [69]:
chunk_list = []
chunk_metadata = []
for chunk in doc:
    chunk_list.append(chunk.page_content)
    chunk_metadata.append(chunk.metadata)

In [70]:
len(chunk_metadata)

160

In [71]:
len(chunk_list)

160

In [73]:
for chunk, embedding in zip(chunk_list,embedded):
    print(chunk)
    print(embedding,end="\n\n")


Learning OpenCV
Gary Bradski and Adrian Kaehler
Beijing · Cambridge · Farnham · Köln · Sebastopol · Taipei · Tokyo
FM-R4886-AT1.indd   iFM-R4886-AT1.indd   i 9/15/08   4:26:38 PM9/15/08   4:26:38 PM
[-0.028989894315600395, -0.03105894662439823, -0.09450376033782959, 0.022294584661722183, -0.010624706745147705, -0.031197622418403625, 0.09344770759344101, 0.01900545507669449, 0.004986496176570654, -0.06396307796239853, -0.045914653688669205, -0.13800935447216034, -0.0011186772026121616, -0.013954250141978264, -0.007580927573144436, 0.052903179079294205, -0.11315525323152542, -0.0058776261284947395, -0.03251580148935318, -0.08942477405071259, 0.01619209162890911, -0.01796702854335308, 0.05315999314188957, 0.024325959384441376, -0.05949762836098671, 0.014878220856189728, 0.003165309550240636, 0.0012278278591111302, -0.03995658829808235, -0.09374076128005981, -0.02664010599255562, -0.004187717102468014, 0.0976760983467102, -0.06576777994632721, 0.01849433407187462, -0.06464090943336487, -0.

In [74]:
len(embedding_function.embed_query("hii"))

384

In [75]:
from pinecone import Pinecone
pc = Pinecone(api_key="823d885f-e93a-40b7-9290-6e6a2186c599")
index = pc.Index("laminit5")

In [76]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [77]:
z = zip(["id"+str(i) for i in range(519)], embedded, chunk_metadata)

In [78]:
index.upsert(vectors=zip(["id"+str(i) for i in range(0,161)],embedded,chunk_metadata))

{'upserted_count': 160}

In [79]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [80]:
query = embedding_function.embed_query("What is opencv")

In [81]:
len(query)

384

In [84]:
query.

[-0.047301679849624634,
 -0.028015684336423874,
 -0.07236171513795853,
 -0.00801646988838911,
 0.10711796581745148,
 -0.024770738556981087,
 -0.04050205647945404,
 0.02229425124824047,
 0.08616738766431808,
 0.031554318964481354,
 -0.056181877851486206,
 -0.023757997900247574,
 -0.03612950071692467,
 0.026545889675617218,
 -0.04706830158829689,
 -0.01669849269092083,
 -0.017697887495160103,
 0.006305102724581957,
 0.024584829807281494,
 0.04391396418213844,
 -0.06618476659059525,
 0.07558330148458481,
 0.042445991188287735,
 -0.048482321202754974,
 0.00550371827557683,
 0.01342877745628357,
 -0.0485253669321537,
 0.06755658239126205,
 -0.007002630271017551,
 -0.08046184480190277,
 0.0569748654961586,
 -0.008949183858931065,
 0.07458946853876114,
 0.0010531877633184195,
 0.011087357066571712,
 0.03310569375753403,
 -0.06369884312152863,
 -0.035977765917778015,
 -0.1370265632867813,
 -0.007380308583378792,
 -0.008902465924620628,
 0.038320403546094894,
 0.00737327104434371,
 0.0763406008

In [86]:
query2 = embedding_function.embed_query("what is computer vision")
query2

[-0.040049802511930466,
 0.026384525001049042,
 -0.07362324744462967,
 -0.11718709021806717,
 0.03188258782029152,
 0.0054212515242397785,
 0.09515032172203064,
 0.010970302857458591,
 0.04464191198348999,
 -0.029688308015465736,
 -0.05433782562613487,
 -0.04020700976252556,
 0.05100872740149498,
 -0.007581551093608141,
 -0.0848718211054802,
 -0.03233802691102028,
 -0.046450771391391754,
 -0.004959278739988804,
 -0.03243769332766533,
 0.01051696203649044,
 0.01309336069971323,
 -0.05206483602523804,
 -0.07247141003608704,
 -0.050716422498226166,
 0.007329412270337343,
 0.10257109254598618,
 0.034109119325876236,
 -0.03191429004073143,
 -0.04951701685786247,
 -0.02823055163025856,
 -0.033192332834005356,
 0.011644530110061169,
 0.10123185813426971,
 0.059667255729436874,
 -0.035276222974061966,
 0.023050937801599503,
 0.017671426758170128,
 -0.000868974020704627,
 -0.030457202345132828,
 -0.04446079581975937,
 -0.08236654102802277,
 0.04160739481449127,
 -0.0025802399031817913,
 0.01474

In [83]:
index.query(
    query = query,
    top_k=3
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'Content-Length': '90', 'x-pinecone-request-latency-ms': '15', 'x-pinecone-request-id': '8170217072995779189', 'date': 'Thu, 14 Mar 2024 17:34:46 GMT', 'x-envoy-upstream-service-time': '5', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":3,"message":"Cannot provide both 'ID' and 'vector' at the same time","details":[]}
